In [ ]:
library(dplyr)
library(tibble)
library(tidyr)

rm(list=ls())

In [ ]:
source(paste0("/data/repos/actin-personalization/scripts/ncr/ncr_data_exploration_functions.R"))
source(paste0("/data/repos/actin-personalization/scripts/ncr/ncr_patients_like_me_functions.R"))

In [ ]:
ncr <- load_ncr_data_notebook()

In [ ]:
## Collect written treatments of treatment plan
ncr_treatments_written <- collect_all_treatments_written(ncr)

In [ ]:
## Select start and stop interval for every part of the treatment plan, and merge with initial treatment plan and calculate durations
ncr_treatments_start_and_stop <- collect_all_treatments_start_and_end_dates(ncr)

In [ ]:
## Merge written treatments and start/stop treatment dates
ncr_treatment_plan_details <- inner_join(ncr_treatments_written, ncr_treatments_start_and_stop, by=c('key_nkr','key_zid','key_eid'))

In [ ]:
## Add duration as treatment stop date - treatment start date
ncr_treatment_plan_details <- ncr_treatment_plan_details %>%
  add_column(treatment_duration_1 = as.integer(ifelse(ncr_treatment_plan_details$treatment_plan_part_1_stop == "","", as.integer(ncr_treatment_plan_details$treatment_plan_part_1_stop)-as.integer(ncr_treatment_plan_details$treatment_plan_part_1_start)))) %>%
  add_column(treatment_duration_2 = as.integer(ifelse(ncr_treatment_plan_details$treatment_plan_part_2_stop == "","", as.integer(ncr_treatment_plan_details$treatment_plan_part_2_stop)-as.integer(ncr_treatment_plan_details$treatment_plan_part_2_start)))) %>%
  add_column(treatment_duration_3 = as.integer(ifelse(ncr_treatment_plan_details$treatment_plan_part_3_stop == "","", as.integer(ncr_treatment_plan_details$treatment_plan_part_3_stop)-as.integer(ncr_treatment_plan_details$treatment_plan_part_3_start)))) %>%
  add_column(treatment_duration_4 = as.integer(ifelse(ncr_treatment_plan_details$treatment_plan_part_4_stop == "","", as.integer(ncr_treatment_plan_details$treatment_plan_part_4_stop)-as.integer(ncr_treatment_plan_details$treatment_plan_part_4_start)))) %>%
  add_column(treatment_duration_5 = as.integer(ifelse(ncr_treatment_plan_details$treatment_plan_part_5_stop == "","", as.integer(ncr_treatment_plan_details$treatment_plan_part_5_stop)-as.integer(ncr_treatment_plan_details$treatment_plan_part_5_start)))) %>%
  add_column(treatment_duration_6 = as.integer(ifelse(ncr_treatment_plan_details$treatment_plan_part_6_stop == "","", as.integer(ncr_treatment_plan_details$treatment_plan_part_6_stop)-as.integer(ncr_treatment_plan_details$treatment_plan_part_6_start)))) %>%
  add_column(treatment_duration_7 = as.integer(ifelse(ncr_treatment_plan_details$treatment_plan_part_7_stop == "","", as.integer(ncr_treatment_plan_details$treatment_plan_part_7_stop)-as.integer(ncr_treatment_plan_details$treatment_plan_part_7_start))))       

In [ ]:
nrow(ncr_treatments_written)
nrow(ncr_treatment_plan_details)

In [ ]:
## Find similar patients and what type of treatment they received
patient <- read.csv(paste0("/data/actin/patient_characteristics.csv"), sep = ";")

age <- patient$age
who <- patient$who
patient_has_had_surgery <- patient$patient_has_had_surgery
morf_cat <- patient$morf_cat
topo_sublok <- patient$topo_sublok
meta_topo_sublok1 <- c('C480','C481','C482','C488')

ncr_similar_excl_who_and_age <- find_similar_patients_2(ncr_ref_data = ncr, patient_age = age, patient_who = who, patient_morf_cat=morf_cat, patient_topo_sublok=topo_sublok, patient_meta_topo_sublok1=meta_topo_sublok1)
ncr_similar_incl_who_and_age <- find_similar_patients_3(ncr_ref_data = ncr, patient_age = age, patient_who = who, patient_morf_cat=morf_cat, patient_topo_sublok=topo_sublok, patient_meta_topo_sublok1=meta_topo_sublok1)

ncr_similar_with_treatment_excl_who_and_age <- inner_join(ncr_similar_excl_who_and_age, ncr_treatment_plan_details, by=c('key_nkr','key_zid'))
ncr_similar_with_treatment_incl_who_and_age <- inner_join(ncr_similar_incl_who_and_age, ncr_treatment_plan_details, by=c('key_nkr','key_zid'))

ncr_similar_with_treatment_incl_who_and_age %>%
    arrange(key_zid, epis) %>%
    dplyr::filter(chir==0 & mdl_res==0 & rt ==0 & chemort == 0 & hipec == 0 & meta_chir_code1 == "" & meta_rt_code1 == "") %>%
    select(c('key_zid','leeft','epis','morf_cat','meta_topo_sublok1','meta_topo_sublok2','meta_int1','mdl_res','chir','rt','chemort','hipec','chemo','target','tumgericht_ther','treatment_plan_part_1_written','treatment_plan_part_2_written','treatment_plan_part_3_written'))

ncr_similar_with_treatment_excl_who_and_age %>%
    arrange(key_zid, epis) %>%
    dplyr::filter(chir==0 & mdl_res==0 & rt ==0 & chemort == 0 & hipec == 0 & meta_chir_code1 == "" & meta_rt_code1 == "") %>%
    select(c('key_zid','leeft','epis','morf_cat','meta_topo_sublok1','meta_topo_sublok2','meta_int1','mdl_res','chir','rt','chemort','hipec','chemo','target','tumgericht_ther','treatment_plan_part_1_written','treatment_plan_part_2_written','treatment_plan_part_3_written'))


In [ ]:
## Select similar patients in case of HIPEC & certain prior surgeries (WIP)
hipec_int_min <- patient$hipec_int-150 
hipec_int_max <- patient$hipec_int+150
topo_sublok <- patient$topo_sublok
chir_type1 <- patient$chir_type1 

ncr_similar_hipec <- ncr %>%
  dplyr::filter(epis=='VERB') %>%
  dplyr::filter(topo_sublok==topo_sublok) %>%
  dplyr::filter(hipec_int1>hipec_int_min & hipec_int1<hipec_int_max) %>%
  dplyr::filter(meta_topo_sublok1 %in% meta_topo_sublok1 & meta_topo_sublok2=="") %>%
  dplyr::filter(is.na(syst_start_int1)) %>%
  dplyr::filter(rt == 0 & chemort == 0 & chemo == 0 & target == 0 & mdl_res == 0 & meta_rt_code1 == "")

#ncr_similar_hipec %>% 
#select(c('key_zid','leeft','epis','morf_cat','meta_topo_sublok1','meta_topo_sublok2','meta_int1','mdl_res','chir','meta_chir_code1','meta_rt_code1','rt','chemort','hipec','chemo','target','tumgericht_ther'))

ncr_similar_surgery <- ncr %>%
  dplyr::filter(epis=='DIA') %>%
  dplyr::filter(topo_sublok==topo_sublok) %>%
  dplyr::filter(chir_type1==chir_type1 & is.na(chir_type2)) %>%
  dplyr::filter(is.na(syst_start_int1)) %>%
  dplyr::filter(rt == 0 & chemort == 0 & chemo == 0 & target == 0 & mdl_res == 0 & hipec == 0 & meta_chir_code1 == "" & meta_rt_code1 == "")

#ncr_similar_surgery %>% 
#select(c('key_zid','leeft','epis','morf_cat','meta_topo_sublok1','meta_topo_sublok2','meta_int1','mdl_res','chir','meta_chir_code1','meta_rt_code1','rt','chemort','hipec','chemo','target','tumgericht_ther'))

ncr_similar_pts <- ncr %>%
dplyr::filter(key_zid %in% ncr_similar_hipec$key_zid) %>%
dplyr::filter(key_zid %in% ncr_similar_surgery$key_zid)

#ncr_similar_pts %>% 
#    arrange(key_zid, epis) %>%
#    select(c('key_zid','epis','mdl_res','chir','chir_type1','chir_int1','meta_topo_sublok1','meta_topo_sublok2','meta_int1','rt','chemort','syst_start_int1','syst_start_int2','syst_start_int3','hipec','hipec_int1'))

#what is known about similar pts
ncr_similar_pts_hipec_date <- ncr_similar_hipec %>% select('key_zid','hipec_int1')
ncr_similar_pts_died <- ncr_similar_pts %>% dplyr::filter(epis=='DIA' & vit_stat==1) %>% select('key_zid','vit_stat_int')
ncr_similar_pts_alive <- ncr_similar_pts %>% dplyr::filter(epis=='DIA' & vit_stat==0) %>% select('key_zid','vit_stat_int')
ncr_similar_pts_died_date <- inner_join(ncr_similar_pts_died, ncr_similar_pts_hipec_date, by=c('key_zid'))
ncr_similar_pts_alive_date <- inner_join(ncr_similar_pts_alive, ncr_similar_pts_hipec_date, by=c('key_zid'))

ncr_similar_pts_died_date$hipec_vit_stat_int <- ncr_similar_pts_died_date$vit_stat_int - ncr_similar_pts_died_date$hipec_int1
ncr_similar_pts_alive_date$hipec_vit_stat_int <- ncr_similar_pts_alive_date$vit_stat_int - ncr_similar_pts_alive_date$hipec_int1

ncr_similar_pts_died %>% nrow()
ncr_similar_pts_alive %>% nrow() 

paste0("Median OS after hipec date for patients who died: ",median(as.numeric(ncr_similar_pts_died_date$hipec_vit_stat_int)))
paste0("Median vit stat int after hipec date for patients who are still alive: ",median(as.numeric(ncr_similar_pts_alive_date$hipec_vit_stat_int)))

In [ ]:
## Calculate OS (WIP)

ncr_similar_alive <- ncr_similar %>% dplyr::filter(vit_stat==0)
ncr_similar_died <- ncr_similar %>% dplyr::filter(vit_stat==1)
ncr_similar_died_treated <- ncr_similar %>% dplyr::filter(tumgericht_ther==1) %>% dplyr::filter(chemo == 2 | target == 2) %>% dplyr::filter(vit_stat==1)
ncr_similar_died_no_treatment <- ncr_similar %>% dplyr::filter(tumgericht_ther==0) %>% dplyr::filter(vit_stat==1)

ncr_similar_alive %>% nrow()
ncr_similar_died %>% nrow()
ncr_similar_died_treated %>% nrow()
ncr_similar_died_no_treatment %>% nrow()

paste0("Median OS of all similar patients who died: ",median(as.numeric(ncr_similar_died$vit_stat_int)))
paste0("Median OS of all similar patients who died with chemo or targeted systemic therapy: ",median(as.numeric(ncr_similar_died_treated$vit_stat_int)))
paste0("Median OS of all similar patients who died without treatment: ",median(as.numeric(ncr_similar_died_no_treatment$vit_stat_int)))